제너레이터는 전통적인 언어와 파이썬을 구분 짓는 또 다른 특징적인 기능이다. 제너레이터를 사용해 이상적으로 문제를 해결하는 방법과 제너레이터(또는 이터러블)를 파이썬스럽게 구현할 수 있는지 알아본다.

이터레이터 패턴을 따르면 왜 언어에서 자동으로 반복을 지원하게 되는지도 알게 될 것이다. 이 개념에서 출발하여 제너레이터가 어떻게 코루틴이나 비동기 프로그래밍 같은 기능을 지원하기 위한 기본 기능이 되었는지 살펴볼 것이다.

* 프로그램의 성능을 향상시키는 제너레이터 만들기
* 이터레이터가 파이썬에 어떻게 (특히 이터레이터 패턴을 사용하여) 완전히 통합되었는지 확인
* 이터레이션 문제를 이상적으로 해결하는 방법
* 제너레이터가 어떻게 코루틴과 비동기 프로그래밍의 기반이 되는 역할을 하는지 확인 
* 코루틴을 지원하기 위한 yield from, await, async def와 같은 문법의 세부 기능 확인

## 제너레이터 만들기

제너레이터는 파이썬에서 고성능이면서도 메모리를 적게 사용하는 반복을 위한 방법으로(PEP-255) 아주 오래 전 2001년에 소개되었다.

제너레이터는 한 번에 하나씩 구성요소를 반환해주는 이터러블을 생성해주는 객체이다. 제너레이터를 사용하는 주요 목적은 메모리를 절약하는 것이다. 거대한 요소를 한꺼번에 메모리에 저장하는 대신 특정 요소를 어떻게 만드는지 아는 객체를 만들어서 필요할 때마다 하나씩만 가져오는 것이다.

이 기능은 하스켈과 같은 다른 함수형 프로그래밍 언어가 제공하는 것과 비슷한 방식으로 게으른 연산(lazy computation)을 통해 무거운 객체를 사용할 수 있도록 한다. 게으른 연산의 특성을 가졌기 때문에 무한 시퀀스를 사용할 수도 있다.

### 제너레이터 개요

먼저 예제를 살펴보자. 지금 하려는 것은 대규모의 구매 정보에서 최저 판매가, 최고 판매가, 평균 판매가를 구하는 것이다.

문제의 단순화를 위해 두 개의 필드만 있는 CSV 파일이 있다고 가정해보자.

모든 구매 정보를 받아 필요한 지표를 구해주는 객체를 만들어보자. 최솟값이나 최댓값 같은 지표는 min(), max() 같은 내장 함수를 사용하여 쉽게 구할 수 있다. 그러나 어떤 지표는 단번에 구할 수 없고 모든 구매 이력을 반복해야만 한다.

지표를 구하는 코드 자체는 간단하다. for 루프의 각 단계에서 각 지표를 업데이트하기만 하면 된다. 일단 다음처럼 간단하게 구현을 하고 제너레이터에 대한 학습을 한 뒤에 훨씬 간단하고 깔끔한 형태의 구현을 다시 해볼 것이다.

In [84]:
class PurchasesStats:
    def __init__(self, purchases):
        self.purchases = iter(purchases)
        self.min_price: float = None
        self.max_price: float = None
        self._total_purchases_price: float = 0.0
        self._total_purchases = 0
        self._initialize()
    
    def _initialize(self):
        try:
            first_value = next(self.purchases)
        except:
            raise ValueError("no values provided")
        
        self.min_price = self.max_price = first_value
        self._update_avg(first_value)
    
    def process(self):
        for purchase_value in self.purchases:
            self._update_min(purchase_value)
            self._update_max(purchase_value)
            self._update_avg(purchase_value)
        return self
    
    def _update_min(self, new_value: float):
        if new_value < self.min_price:
            self.min_price = new_value
    
    def _update_max(self, new_value: float):
        if new_value > self.max_price:
            self.max_price = new_value
    
    @property
    def avg_price(self):
        return self._total_purchases_price / self._total_purchases
    
    def _update_avg(self, new_value: float):
        self._total_purchases_price += new_value
        self._total_purchases += 1
        
    def __str__(self):
        return (
            f"{self.__class__.__name__}({self.min_price}), "
            f"{self.max_price}, {self.avg_price})"
        )

이 객체는 모든 구매 정보를 받아서 필요한 계산을 한다. 이제 이 모든 정보를 로드해서 어딘가에 담아서 반환해주는 함수를 만들어보자. 다음은 첫 번째 버전이다.

In [49]:
def _load_purchases(filename):
    purchases = []
    with open(filname) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            purchases.append(float(price_raw))
    return purchases

이 코드는 정상적인 결과를 반환한다. 파일에서 모든 정보를 읽어서 리스트에 저장한다. 그러나 성능에 문제가 있다. 파일에 상당히 많은 데ㅣ터가 있다면 로드하는데 시간이 오래 걸리고 메인 메모리에 담지 못할 만큼 큰 데이터일 수도 있다.

그런데 앞서 작성한 코드를 살펴보면 한 번에 하나의 데이터만을 사용하고 있다는 것 알 수 있다. 그렇다면 굳이 파일의 모든 데이터를 한 번에 모두 읽어 와서 메모리에 보관해야 할 이유가 무엇일까? 뭔가 더 좋은 해결책이 있을 것이다.

해결책은 제너레이터를 만드는 것이다. 파일의 전체 내용을 리스트에 보관하는 대신에 필요한 값만 그때그때 가져오는 것이다. 다음과 같이 코드를 수정한다.

In [50]:
def load_purchases(filename):
    with open(filename) as f:
        for line in f:
            *_, price_raw = line.partition(",")
            yield float(price_raw)

이렇게 수정하면 메모리 사용량이 급격하게 떨어지는 것을 볼 수 있다. 결과를 담을 리스트가 필요 없어졌으며 return 문 또한 사라졌다.

이 경우 load_purchases 함수를 제너레이터 함수 또는 단순히 제너레이터라고 부른다.

파이썬에서 어떤 함수라도 yield 키워드를 사용하면 제너레이터 함수가 된다. yeild가 포함된 이 함수를 호출하면 제너레이터의 인스턴스를 만든다.

모든 제너레이터 객체는 이터러블이다. 이터러블에 대해서는 뒤에 자세히 알아본다.

이터러블은 for 루프와 함께 사용할 수 있다는 정도만 알아두자. 여기서 주목할 점은 이 함수의 사용 코드가 그대로라는 점이다. 코드를 수정한 뒤에도 for 루프를 사용하여 지표를 계산하는 코드는 그대로이다. 

이터러블을 사용하면 for 루프의 다형성을 보장하는 이와 같은 강력한 추상화가 가능하다. 이터러블 인터페이스를 따르면 투명하게 객체의 요소를 반복하는 것이 가능하다.

### 제너레이터 표현식

제너레이터를 사용하면 많은 메모리를 절약할 수 있다. 또한 제너레이터는 이터레이터이므로 리스트나 튜플, 세트처럼 많은 메모리를 필요로 하는 이터러블이나 컨테이너의 대안이 될 수 있다. 

컴프리헨션에 의해 정의될 수 있는 리스트나 세트, 사전처럼 제너레이터도 제너레이터 표현식으로 정의할 수 있따. 물론 제너레이터 표현시을 제너레이터 컴프리헨션으로 불러야 한다는 주장도 있다. 

같은 방법으로 리스트 컴프리헨셩을 사용해볼 것이다. 대괄호를 괄호로 교체하면 표현식의 결과로부터 제너레이터가 생성된다. 제너레이터 표현식은 sum()이나 max()와 같이 이터러블 연산이 가능한 함수에 직접 전달할 수도 있다.

In [51]:
[x**2 for x in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [52]:
(x**2 for x in range(10))

<generator object <genexpr> at 0x07592030>

In [53]:
sum(x**2 for x in range(10))

285

## 이상적인 반복

파이썬에서 반복을 할 때 유용하게 사용할 수 있는 관용적인 코드를 살펴볼 것이다. 이러한 코드 예제는(특히 제너레이터 표현식을 알게 된 뒤에는) 제너레이터를 활용한는 방법을 익힐 수 있게 해주고 제너레이터와 관련된 전현적인 문제를 해결하는 방법을 알려준다. 

### 관용적인 반복 코드

우리는 이미 내장 함수인 enumerate()에 익숙하다. 이 함수는 이터러블을 입력 받아서 인덱스 번호와 원본의 원소를 튜플 형태로 변환하여 enumerate 객체를 반환한다.

In [54]:
list(enumerate('abcdef'))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

좀 더 저수준에서 이와 유사한 객체를 만들어볼 것이다. 이 객체는 단순히 시작 값을 입력하면 무한 시퀀스를 만드는 역할을 한다.

다음과 같이 간단하게 만들 수 있다. 객체의 next() 함수를 호출할 때마다 다음 시퀀스 값을 무제한 출력해준다.

In [55]:
class NumberSequence:
    def __init__(self, start=0):
        self.current = start
    
    def next(self):
        current = self.current
        self.current += 1
        return current

이 인터페이스에 기반을 두어 클라이언트를 작성하면 명시적으로 next() 함수를 호출해야 한다.

In [56]:
seq = NumberSequence()
seq.next()

0

In [57]:
seq.next()

1

In [58]:
seq2 = NumberSequence(10)
seq2.next()

10

그러나 이 코드로 enumerate() 함수를 사용하도록 재작성할 수는 없다. 왜냐하면 일반 파이썬의 for 루프를 사용하기 위한 인터페이스를 지원하지 않기 때문이다. 이는 또한 이터러블 형태의 파라미터로는 사용할 수 없다는 것을 뜻한다. 다음 코드를 살펴보자.

In [59]:
list(zip(NumberSequence(), "abcdef"))

TypeError: zip argument #1 must support iteration

문제는 Numbersequence가 반복을 지원하지 않는다는 것이다. 이 문제를 해결하려면 \__iter__() 매직 메서드를 구현하여 객체가 반복 가능하게 만들어야한다. 또한 next() 메서드를 수정하여 \__next__ 매직 메서드를 구현하면 객체는 이터레이터가 된다.

In [60]:
class SequenceofNumbers:
    def __init__(self, start=0):
        self.current = start
    
    def __next__(self):
        current = self.current
        self.current += 1
        return current
    
    def __iter__(self):
        return self

이렇게 하면 요소를 반복할 수 있을 뿐 아니라, .next() 메서드를 호출할 필요도 없다. 왜냐하면 \__next__() 메서드를 구현헀으므로 next() 내장 함수를 사용할 수 있기 때문이다.

In [61]:
list(zip(SequenceofNumbers(), "abcdef"))

[(0, 'a'), (1, 'b'), (2, 'c'), (3, 'd'), (4, 'e'), (5, 'f')]

In [62]:
seq = SequenceofNumbers(100)

In [63]:
next(seq)

100

In [64]:
next(seq)

101

### next() 함수

next() 내장 함수는 이터러블을 다음 요소로 이동시키고 기존의 값을 반환한다.

In [65]:
word = iter('hello')
next(word)

'h'

In [66]:
next(word)

'e'

In [67]:
next(word)
next(word)
next(word)

'o'

In [68]:
next(word)

StopIteration: 

이터레이터가 더 이상의 값을 가지고 있지 않다면 StopIteration 예외가 발생한다.

이 예외는 반복이 끝났음을 나타내며 사용할 수 있는 요소가 더 이상  없음을 나타낸다.

이 문제를 해결하고 싶다면 StopIteration 예외를 캐치하는 것 외에도 next() 함수의 두 번째 파라미터에 기본 값을 제공할 수도 있다. 이 값을 제공하면 StopIertation을 발생시키는 대신 기본 값을 반환한다.

In [69]:
next(word, "default value")

'default value'

### 제너레이터 사용하기

앞의 코드는 제너레이터를 사용하여 훨씬 간단하게 작성할 수 있다. 제너레이터를 사용하면 클래스를 만드는 대신 다음과 같이 필요한 값을 yield하는 함수를 만들면 된다.

In [70]:
def seqeunce(start=0):
    while True:
        yield start
        start += 1

함수의 본문에 있는 yield 키워드가 해당 함수를 제너레이터로 만들어 준다는 것에 유의하자. 이 함수는 제너레이터이기 때문에 위처럼 무한 루프를 사용해도 완벽하게 안전하다. 제너레이터 함수가 호출되면 yield 문장을 만나기 전까지 실행된다. 그리고 값을 생성하고 그 자리에서 멈춘다.

In [71]:
seq = seqeunce(10)

In [72]:
next(seq)

10

In [73]:
next(seq)

11

In [74]:
next(seq)

12

In [75]:
list(zip(seq, "abcdef"))

[(13, 'a'), (14, 'b'), (15, 'c'), (16, 'd'), (17, 'e'), (18, 'f')]

In [76]:
def seq(start=0):
    while True:
        yield start
        start += 1
        if start == 3:
            break

In [77]:
seq = seq()

In [78]:
next(seq)

0

In [79]:
next(seq)

1

In [80]:
next(seq)

2

In [81]:
next(seq, 10)

10

### Itertools 

이터러블로 작업하면 코드가 파이썬 자체와 더 잘 어울리는 장점이 있다. 왜냐하면 이터레이션이 언어의 중요한 컴포넌트이기 때문이다. 또란 itertools 모듈(ITER-01)을 사용하면 그 기능을 온전히 활용할 수 있다. 사실 방금 사용한 sequence() 제너레이터는 itertools.count()와 상당히 유사하다. 그러나 itertools 모듈에는 여러 추가 기능이 있다.

이터레이터, 제너레이터, itertools와 관련하여 가장 멋진 것 중에 하나는 이들을 서로 연결하여 새로운 객체를 만들 수 있다는 것이다.

예를 들어 처음에 예제로 돌아가서 구매 이력에서 지표를 계산하는 과정을 다시 살펴보자. 만약 특정 기준을 넘은 값에 대해서만 연산을 하려면 어떻게 해야 할까? 가장 간단한 방법은 while 문 안에 조건을 추가하는 것이다.

In [82]:
def process(self):
    for purchase in self.purchases:
        if purchase > 1000.0:
            ...

그러나 이 방법은 파이썬스럽지 않을 뿐 아니라 너무 엄격하다. 너무 엄격하다는 것은 나쁜 코드를 의미한다. 이것은 수정 사항을 잘 반영할 수 없다. 만약 기준 수치가 변경된다면 파라미터로 전달해야 할까? 만약 파라미터가 둘 이상 필요하다면 어떻게 할까? 만약 조건이 특정 기준 이하가 되면 어떻게 할까? 람다 함수를 사용해야 할까?

사실 이번 객체가 이 질문에 대한 답변을 할 필요는 없다. 이 객체의 고유 책임은 구매 이력에 대해 잘 정의된 지표 값을 계산하고 출력하는 것뿐이다. 이러한 요구사항을 이번 객체에 반여하는 것은 큰 실수이다. 다시 한 번 말하지만 클린 코드는 융통성이 잇어야 하고 외부 요인에 결합력이 높아서는 안 된다. 이러한 요구사항은 다른 곳에서 해결되어야 한다.

이 객체는 클라이언트의 요구로부터 독립되어야 한다. 이 클래스의 책임이 작을수록 클라이언트는 재사용성이 높아지믈 보다 유용하게 된다.

코드를 수정하는 대신 그대로 유지하고 클라이언트 클래스 요구사항이 무엇이든 그에 맞게 필터링하여 새로운 데이터를 만든다고 가정하자.

예를 들어 1,000개 넘게 구매한 이겨의 처음 10개만 처리하려고 하면 다음과 같이 하면 된다.

In [93]:
from itertools import islice
purchases = [995+i for i in range(20)] # 테스트를 위해 생성함 이 객체는 메모리에 올라감 
purchases = islice(filter(lambda p:p > 1000.0, purchases), 10)
stats = PurchasesStats(purchases).process()
print(stats)

PurchasesStats(1001), 1010, 1005.5)


이런 식으로 필터링을 해도 메모리의 손해는 없다. 왜냐하면 모든 것이 제너레이터이므로 게으르게 평가된다. 즉 마치 전체에서 필터링한 값으로 연산을 한 것처럼 보이지만, 실제로는 하나씩 가져와서 모든 것을 메모리에 올릴 필요가 없는 것이다.

### 이터레이터를 사용한 코드 간소화

지금까지 이터레이터 또는 itertools 모듈을 활용해 코드를 개선하는 것을 살펴보았다. 각 사례의 최적화 방법에 대한 결론도 추론할 수 있다.

### 여러 번 반복하기

이제 이터레이터에 대해 좀 더 깊이 있게 살펴보았고 itertools 모듈까지 확인하였으므로 이 장의 처음에 살펴보았던 예제를 훨씬 간소화할 수 있다.

In [106]:
import itertools

def process_purchases(purchases):
    min_, max_, avg = itertools.tee(purchases, 3)
    print(min_, max_, avg)
    return min(min_), max(max_), median(avg)

In [107]:
purchases = [995+i for i in range(20)] # 테스트를 위해 생성함 이 객체는 메모리에 올라감 
print(process_purchases(purchases))

<itertools._tee object at 0x071A05F8> <itertools._tee object at 0x07493328> <itertools._tee object at 0x07493AF8>
(995, 1014, 1004.5)


itertools.tee*는 원래의 이터러블을 세 개의 새로운 이터러블로 분할한다. 그리고 구매 이력을 세 번 반복할 필요없이 분할된 이터러블을 사용해 필요한 연산을 할 것이다. 

### 중첩 루프

경우에 따라 1차원 이상을 반보개서 값을 찾아야 할 수 있다. 가장 쉽게 해결하는 방법으로 중첩 루프가 떠오른다. 값을 찾으면 순환을 멈추고 break 키워드를 호출해야 하는데 이런 경우 한 단계가 아니라 두 단계 이상을 벗언나야 하므로 정상적으로 동작하지 않는다.

플래그나 예외를 발생시켜야 할까? 아니다. 플래그와 마찬가지이고 오히려 더 나쁘다. 왜냐하면 예외는 로직을 제어하기 위한 수단이 아니기 때문이다. 코드를 잘게 나누어 함수에서 반환해야 할까? 비슷했지만 완벽한 방법은 아니다.

가장 좋은 방법은 가능하면 중첩을 풀어서 1차원 루프로 만드는 것이다.

다음은 피해야 할 코드이다.

In [108]:
def search_nested_bad(array, desired_value):
    coords = None
    for i, row in enumrate(array):
        for j, cell in enumerate(row):
            if cell == desired_value:
                coords = (i, j)
                break
        if coords is not None:
            break
    
    if coords is None:
        raise ValueError(f"{desired_value} not found")
    
    logger.info("[%i, %i]에서 값 %r 찾음", *coords, desired_value)
    return coords

다음은 종료 플래그를 사용하지 않은 보다 간단하고 컴팩트한 형태의 예이다.

In [109]:
def _iterate_array2d(array2d):
    for i, row in enumerate(array2d):
        for j, cell in enumerate(row):
            yield (i, j), cell

def search_nested(array, desired_value):
    try:
        coord = next(coord for (coord, cell) in _iterate_array2d(array) if cell == desired_value)
    except StopIteration:
        raise ValueError("{desired_value} not found")
        
    logger.info("[%i, %i]에서 값 %r 찾음", *coords, desired_value)
    return coord

보조 제너레이터가 어떻게 반복을 추상화했는지 살펴볼 수 있다. 지금은 2차원 배열을 사용했으나 나중에 더 많은 차원의 배열을 사용하는 경우에도 클라이언트는 그서에 대해 알 필요가 없이 기존 코드를 그대로 사용하면 된다. 이것이 다음 섹션에서 설명할 이터레이터 디자인 패턴의 본질이다. 파이썬은 이터레이터 객체를 지원하므로 자동으로 투명해진다.

* 최대한 중첩 루프를 제거하고 추상화하여 반복을 단순화한다.

### 파이썬의 이터레이터 패턴

파이썬에서의 반복을 좀 더 자세히 이해하기 위해 제너레이터로부터 약간 벗어나볼 것이다. 제너레이터는 이터러블 객체의 특별한 경우이지만 파이썬의 반복은 제너레이터 이상의 것으로 훌륭한 이터러블 객체를 만들게 되면 보다 효율적이고 컴팩트하고 가독성이 높은 코드를 작성할 수 있게 된다.

앞의 코드에서는 이터러블 객체이면서 또한 이터레이터인 객체를 살펴보았다. 이터레이터는 \__iter__()와 \__next__() 매직 메서드를 구현한 객체이다. 일반적으로 이렇게 구현을 하지만 엄밀히 말하면 항상 이 두 가지를 꼭 구현해야 하는 것은 아니다. 여기서는 \__iter__를 구현한 이터러블 객체와 \__next__를 구현한 이터레이터 객체를 비교해볼 것이다.

또한 시퀀스와 컨테이너 객체의 반복에 대해서도 다룰 것이다.

### 이터레이션 인터페이스

이터러블은 반복을 지원하는 객체로 크게 보면 아무 문제없이 for ... in ... 루프를 실행할 수 있다는 것을 뜻한다. 그러나 이터러블과 이터레이터는 다르다.

일반적으로 이터러블은 반복할 수 있는 어떤 것으로 실제 반복을 할 때는 이터레이터를 사용한다. 즉 \__iter__ 매직 메서드를 통해 이터레이터를 반환하고, \__next__ 매직 메서드를 통해 반복 로직을 구현하는 것이다.

이터레이터는 단지 내장 next() 함수 호출 시 일련의 값에 대해 한 번에 하나씩만 어떻게 생성하는지 알고 있는 객체이다. 이터레이터를 호출하지 않은 상태에서 다음 값을 요청 받기 전까지 그저 얼어있는 상태일 뿐이다. 이러한 의미에서 모든 제너레이터는 이터레이터이다. 

|파이썬 개념|매직 메서드|비고|
|------------------|--------------------|------|
|이터러블| \__iter__ | 이터레이터와 함께 반복 로직을 만든다. 이것을 구현한 객체는 for ... in ... 구문에서 사용할 수 있다.|
|이터레이터| \__next__ | 한 번에 하나씩 값을 생산하는 로직을 정의한다. 더 이상 생산할 값이 없을 경우 StopIteration 예외를 발생시킨다. 내장 next() 함수를 사용해 사나씩 값을 읽어 올 수 있다.|

다음 코드는 이터러블하지 않은 이터레이터 객체의 예이다. 이것은 오직 한 번에 하나씩 값을 가져올 수만 있다. 여기서 sequence는 잠시 후 살펴볼 파이썬의 시퀀스가 아니고 일련의 연속된 숫자를 나타낸다.

In [114]:
class SequenceIterator:
    def __init__(self, start=0, step=1):
        self.current = start
        self.step = step
    
    def __next__(self):
        value = self.current
        self.current += self.step
        return value

시퀀스에서 하나씩 값을 가져올 수 있지만 반복할 수는 없다(무한 루프를 유발하므로 어쩌면 다행이다).

In [115]:
si = SequenceIterator(1, 2)
next(si)

1

In [116]:
next(si)

3

In [117]:
next(si)

5

In [118]:
for _ in SequenceIterator(): pass

TypeError: 'SequenceIterator' object is not iterable

객체가 \__iter__() 메서드를 구현하지 않았으므로 에러가 발생하는 것이 당연하다.

단지 설명을 하자면 반복을 다른 객체로 분리할 수도 있다.

### 이터러블이 가능한 시퀀스 객체

앞에서 보았듯이 \__iter__() 매직 메서드를 구현한 객체는 for 루프에서 사용할 수 있다. 이것은 큰 특징이지만 꼭 이런 형태여야만 반복이 가능한 것은 아니다. 파이썬이 for 루프를 만나면 객체가 \__iter__를 구현했는지 확인하고 있으면 그것을 사용한다. 그러나 없을 경우는 다른 대비 옵션을 가동한다.

객체가 시퀀스인 경우(즉 \__getitem__()과 \__len__() 매직 메서드를 구현한 경우)도 반복 가능하다. 이 경우 인터프리터는 IndexError 예외가 발생할 때까지 순서대로 값을 제공한다. 

IdexError는 StopIteration과 유사하게 반복에 대한 중지를 알리는 역할을 한다.

이러한 동작을 확인하기 위해 특정 숫자 범위에 대해 map()을 구현한 시퀀스 객체를 살펴보자.

In [125]:
class MappedRange:
    """특정 숫자 범위에 대해 맵으로 변환"""
    
    def __init__(self, transformation, start, end):
        self._transformation = transformation
        self._wrapped = range(start, end)
        
    def __getitem__(self, index):
        value = self._wrapped.__getitem__(index)
        result = self._transformation(value)
        print("Index %d: %s" %(index, result))
        return result
    
    def __len__(self):
        return len(self._wrapped)

이 예제는 오직 일반 for 루프를 사용해 반복 가능하다는 것을 보여주기 위한 것이다. \__getitem__ 메서드에서는 객체가 반복하는 동안 어떤 값이 전달되었는지 확인하기 위해 로그를 출력한다.

In [126]:
mr = MappedRange(abs, -10, 5)

In [127]:
mr[0]

Index 0: 10


10

In [128]:
mr[-1]

Index -1: 4


4

In [129]:
for i in mr:
    print(i)

Index 0: 10
10
Index 1: 9
9
Index 2: 8
8
Index 3: 7
7
Index 4: 6
6
Index 5: 5
5
Index 6: 4
4
Index 7: 3
3
Index 8: 2
2
Index 9: 1
1
Index 10: 0
0
Index 11: 1
1
Index 12: 2
2
Index 13: 3
3
Index 14: 4
4


In [130]:
list(mr)

Index 0: 10
Index 1: 9
Index 2: 8
Index 3: 7
Index 4: 6
Index 5: 5
Index 6: 4
Index 7: 3
Index 8: 2
Index 9: 1
Index 10: 0
Index 11: 1
Index 12: 2
Index 13: 3
Index 14: 4


[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4]

In [131]:
mr

다만 이러한 방법이 있다는 것을 알아두는 것은 좋지만 객체가 \__iter__를 구현하지 않았을 때 동작하는 대비책임에 주의하자. 따라서 대부분의 경우 단순히 반복 가능한 객체를 만드는 것이 아니라 적절한 시퀀스를 만들어 해결하는 것이 바람직하다. 

## 코루틴(coroutine)

이미 알고 있는 것처럼 제너레이터는 반복 가능한 객체로 \__iter__()와 \__next__()를 구현한다. 이를 사용한 프로토콜은 파이썬에 의해 자동으로 제공되므로 제너레이터 객체를 사용하면 next() 함수를 통해 다음 요소로 반복 또는 이동이 가능하다.

이러한 기본 기능 외에도 제너레이터를 코루틴(PEP-342)으로 활용할 수 있다. 여기서는 비동기 프로그래밍의 기초를 지원하기 위해 제너레이터가 어떻게 코루틴으로 진화하는지 살펴보고, 다음 섹션에서 비동기 기능을 위한 파이썬의 새로운 기능과 문법에 대해서 살펴보겠다. 코루틴을 지원하기 위해 PEP-342에 추가된 기본 메서드는 다음과 같다.

* .close()
* .throw(ex_type[, ex_value[, ex_traceback]])
* .send(value)

### 제너레이터 인터페이스의 메서드

이 섹션에서는 앞서 언급한 각각의 메서드가 무엇인지, 어떻게 작동하는지 그리고 어떻게 사용되는지 알아볼 것이다. 메서드의 사용방법을 이해한 뒤에 간단한 코루틴을 만들어볼 것이다.

그리고 코루틴의 고급 사용법과 서브 제너레이터(코루틴)에 위임을 통해 리팩토링하는 방법, 여러 코루틴을 조합하는 방법에 대해서도 알아볼 것이다.

### close()

이 메서드를 호출하면 제너레이터에서 GeneratorExit 예외가 발생한다. 이 예외를 따로 처리하지 않으면 제너레이터가 더 이상 값을 생성하지 않으며 반복이 중지된다.

이 예외는 종료 상태를 지정하는데 사용될 수 있다. 코루틴이 일종의 자원 관리를 하는 경우 이 예외를 통해서 코루틴이 보유한 모든 자원을 ㅎ제할 수 있다. 일반적으로 컨텍스트 관리자를 사용하거나 finally 블록에 코드를 배치하는 것과 비슷하지만 이 예외를 사용하면 보다 명확하게 처리할 수 있다.

다음 예제는 코루틴을 사용하여 데이터베이스 연결을 유지한 상태에서 한 번에 모든 레코드를 읽는 대신에 특정 크기의 페이지를 스트리밍한다.

In [132]:
def stream_db_records(db_handler):
    try:
        while True:
            yield db_handler.read_n_records(10)
    except GeneratorExit:
        db_handler.close()

제너레이터를 호출할 때마다 데이터에비스 핸들러에서 얻은 10 개의 레코드를 반환하고, 명시적으로 반복을 끝내고 close()를 호출하면 데이터베이스 연결도 함께 종료한다.

In [134]:
streamer = stream_db_records(DBHandler('testdb'))
next(streamer)
next(streamer)
streamer.close()

제너레이터에서 작업을 종료할 떄는 close() 메서드를 사용한다.

### throw(ex_type[, ex_value[, ex_traceback]])

이 메서드는 현재 제너레이터가 중단된 위치에서 예외를 던진다. 제너레이터가 예외를 처리했으면 해당 except 절에 있는 코드가 호출되고, 예외를 처리하지 않았으면 예외가 호출자에게 전파된다. 

여기서는 코루틴이 예외를 처리했을 때와 그렇지 않을 때의 차이를 설명하기 위해 코드를 약간 수정했다.

In [136]:
class CustomException(Exception):
    pass
    
    def stream_data(db_handler):
        while True:
            try:
                yield db_handler.read_n_records(10)
            except CustomException as e:
                logger.warning("처리 가능한 에러 %r, 계속 진행", e)
                print('처리 가능한 에러 %r, 계속 진행' %(e))
                
            except Exception as e:
                db_handler.close()
                break

이제 CusmtomException을 처리하고 있으며 이 예외가 발생한 경우 제너레이터는 INFO 레벨의 메세지를 기록한다. 물론 비즈니스 로직에 따라 다르게 구현할 수 있다. 그리고 다음 yield 구문으로 이동하여 데이터베이스에서 다시 데이터를 가져온다.

이 예제는 모든 예외를 처리하고 있지만 마지막 블록이 없으면 제너레이터가 중지된 행에서 예외가 호출자에게 전파되고 제너레이터는 중지된다.

### send(value)

앞의 예제에서는 데이터베이스 레코드를 조회하는 간단한 제너레이터를 만들고 반복을 끝낼 때 데이터베이스 리소스를 해제했다. 이것은 제너레이터가 제공하는 close 메서드를 사용하는 좋은 예제이지만 코루틴으로 보다 많은 일을 할 수 있다.

현재 제너레이터의 주요 기능은 고정된 수의 레코드를 읽는 것이다. 이제 읽어올 개수를 파라미터로 받아서 호출하도록 수정해보자. 안타깝게도 next() 함수는 이러한 옵션을 제공하지 않는다. 이럴 떄 send() 메서드를 사용하면 된다. 

In [137]:
def stream_db_records(db_handler):
    retrieved_data = None
    previous_page_size = 10
    try:
        while True:
            page_size = yield retrieved_data
            if page_size is None:
                page_size = previous_page_size
            previous_page_size = page_size
            
            retrieved_data = db_handler.read_n_records(page_size)
    except GeneratorExit:
        db_handler.close()

이제 send() 메서드를 통해 인자 값을 전달할 수 있다. 사실 이 메서드는 제너레이터와 코루틴을 구분하는 기준이 된다. send() 메서드를 사용했다는 것은 yield 키워드가 할당 구문의 오른쪽에 나오게 되고 인자 값을 받아서 다른 곳에 할당할 수 있음을 뜻한다.

코루틴에서는 일반적으로 다음과 같은 형태로 yield 키워드를 사용한다.

In [139]:
receice = yield produced

호출자는 next() 메서드를 호출하여 다음 라운드가 되었을 때 값을 가져올 수 있다. 다른 하나는 거꾸로 호출자로부터 send() 메서드를 통해 전달된 produced 값을 받는 것이다. 이렇게 입력된 값은 receive 변수에 할당된다.